In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.distributions.normal import Normal
from torch.autograd import Variable
    
def softplus(rho):
    # convert from rho -> sigma
    return torch.log(1 + torch.exp(rho))     

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_weights = 0
        
        self.weight = None # Parameter(torch.FloatTensor(in_features, out_features))
        self.num_weights += in_features * out_features
        
        if bias:
            self.bias = None # Parameter(torch.FloatTensor(out_features))
            self.num_weights += out_features
        else:
            self.register_parameter('bias', None)
                    
    def init_weights(self, weights):
        self.weight = weights[:self.in_features * self.out_features].reshape(self.in_features, self.out_features)
        if self.bias:
            self.bias = weights[self.in_features * self.out_features:]
        
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.layers = [self.gc1, self.gc2]
        self.num_weights = sum(layer.num_weights for layer in self.layers)
        self.dropout = dropout
        
    def kernel(self, x):
        ## Standard RBF kernel: x -> exp(-x*x)
        return torch.exp(-x*x)
    
    def init_params(self):
        return torch.rand(self.num_weights * 2, requires_grad=True) # stored as [mu1, mu2, rho1, rho2]
    
    def init_weights(self, var_params):
        ## Returns log_prob of weights, prior_prob of weights
        w_dist = Normal(var_params[:self.num_weights], softplus(var_params[self.num_weights:]))
        p_dist = Normal(torch.zeros(self.num_weights), torch.ones(self.num_weights) * 5)
        weights = w_dist.rsample()
        
        counter = 0
        for layer in self.layers:
            layer.init_weights(weights[counter:counter+layer.num_weights])
            counter += layer.num_weights
            
        return w_dist.log_prob(weights), p_dist.log_prob(weights)
        
    def forward(self, x, adj):
        x = self.kernel(self.gc1(x, adj))
        # x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)


In [2]:
from dataloader import load_cora, load_adv_data

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_cora()

AttributeError: 'Tensor' object has no attribute 'indices'